In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc

from dataloader.stdata import STData

# sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=400, frameon=False, figsize=(5, 5), facecolor='white')  # low dpi (dots per inch) yields small inline figures
# dic = {0: "WM", 1: "Layer 1", 2: "Layer 2", 3: "Layer 3", 4: "Layer 4", 5: "Layer 5", 6: "Layer 6"}
dic = {0: "0", 1: "1", 2: "2", 3: "3", 4: "4", 5: "5", 6: "6", 7: "7", 8: "8"}

In [ ]:
for dataset in ['200127_15']:
    dataset = str(dataset)
    load_dir = f'results/{dataset}/'
    visium = STData(dataset)
    emb = np.load(load_dir+'emb.npy')
    emb_2d = np.load(load_dir + 'emb_2d.npy')
    pred = np.load(load_dir+'pred_leiden.npy')

    visium.adata.obs['pred'] = np.vectorize(dic.__getitem__)(pred)
    visium.adata.obs['pred'] = visium.adata.obs['pred'].astype('category')
    visium.adata.obsm['emb'] = emb
    visium.adata.obsm['umap'] = emb_2d
    visium.adata.uns['pred_colors'] = ['#2E91E5', '#E15F99', '#1CA71C', '#FB0D0D',
                            '#DA16FF', '#222A2A', '#B68100', '#750D86',
                            '#EB663B', '#511CFB', '#00A08B']
    sc.pp.neighbors(visium.adata, n_neighbors=2, use_rep='emb')
    sc.tl.paga(visium.adata, groups='pred')
    sc.pl.paga_compare(visium.adata, legend_fontsize=0, frameon=False, size=20,
                        legend_fontoutline=0, show=False)
    pl.savefig(load_dir + 'traj.png')
    extent = pl.gca().get_window_extent().transformed(pl.gcf().dpi_scale_trans.inverted())
    pl.gcf().savefig(load_dir+'traj_single.png', bbox_inches=extent)

In [ ]:
# for dataset in [ 151507, 151508, 151509, 151510, 151669, 151670, 151671, 151672, 151673, 151674, 151675, 151676]:
for dataset in [151673]:
    dataset = str(dataset)
    load_dir = f'test_dlpfc/{dataset}/'
    visium = STData(dataset)
    emb = np.load(load_dir+'emb.npy')
    emb_2d = np.load(load_dir + 'emb_2d.npy')
    pred = np.load(load_dir+'pred_main.npy')

    visium.adata.obs['pred'] = np.vectorize(dic.__getitem__)(pred)
    visium.adata.obs['true'] = np.vectorize(dic.__getitem__)(visium.get_label())
    visium.adata.obs['true'] = visium.adata.obs['true'].astype('category')
    visium.adata.obs['pred'] = visium.adata.obs['pred'].astype('category')
    visium.adata.obsm['emb'] = emb
    visium.adata.obsm['umap'] = emb_2d
    visium.adata.uns['true_colors'] = ['#2E91E5', '#E15F99', '#1CA71C', '#FB0D0D', '#DA16FF', '#222A2A', '#B68100']
    sc.pp.neighbors(visium.adata, n_neighbors=10, use_rep='emb')
    sc.tl.paga(visium.adata, groups='true')
    sc.pl.paga_compare(visium.adata, legend_fontsize=0, frameon=False, size=20,
                        legend_fontoutline=0, show=False, threshold=0.3)
    pl.savefig(load_dir + 'traj.png')
    extent = pl.gca().get_window_extent().transformed(pl.gcf().dpi_scale_trans.inverted())
    pl.gcf().savefig(load_dir+'traj_single.png', bbox_inches=extent)
    _ = visium.px_plot_embedding(emb_2d, visium.get_label(), save_path=load_dir+'true_emb.png')